In [27]:
import tensorflow as tf
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

In [28]:
###Preparing data for network training
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [29]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train / 255 
X_test = X_test / 255 
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [30]:
###Function of creating a neural network
def build_model(hp):
    model = tf.keras.models.Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(tf.keras.layers.Dense(units=hp.Int('units_input',    # Fully connected layer with different numbers of neurons
                                   min_value=512,    # minimum number of neurons - 128
                                   max_value=1024,   # maximum number of neurons - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(tf.keras.layers.Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [31]:
###Craete Tuner
tuner = RandomSearch(
    build_model,                 # model creation function
    objective='val_accuracy',    # metric to be optimized - proportion of correct answers on the validation dataset
    max_trials=80,               # maximum number of training starts 
    directory='train_directory'   # directory where trained networks are saved  
    )

INFO:tensorflow:Reloading Oracle from existing project train_directory\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from train_directory\untitled_project\tuner0.json


In [32]:
###Start the selection of hyperparameters
tuner.search_space_summary()

In [33]:
tuner.search(X_train,                  # Data for training
             y_train,                  # Correct answers
             batch_size=256,           # Sample Size
             epochs=20,                # Number of epochs 
             validation_split=0.2,     # The part of the data that will be used for verification
             )

INFO:tensorflow:Oracle triggered exit


In [34]:
###Choosing the best model
tuner.results_summary()

In [35]:
###Get the three best models
models = tuner.get_best_models(num_models=3)

In [36]:
###Evaluate the quality of the model on test data
for model in models:
  model.summary()
  model.evaluate(X_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 928)               728480    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               475648    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 1,209,258
Trainable params: 1,209,258
Non-trainable params: 0
_________________________________________________________________
10000/10000 [==============================] - ETA: 24s - loss: 22.6945 - accuracy: 0.90 - ETA: 1s - loss: 40.4917 - accuracy: 0.8031 - ETA: 0s - loss: 42.4447 - accuracy: 0.793 - ETA: 0s - loss: 42.5137 - accuracy: 0.798 - ETA: 0s - loss: 45.8934 - accuracy: 0.791 - ETA: 0s - loss: 46.3190 - accuracy: 0.792 - ETA: 0s - loss: 48.2122 - accuracy: 0.790 - ETA: 0s - loss: 48.8